In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.preprocessing import LabelEncoder
import math

In [2]:
df = pd.read_csv("/Users/advita/Red-ySetGo/Red-y-Set-Go-Menstrual-Cycle-Tracker/Finalest_data.csv")

In [3]:
def recommend_exercise_type(cycle_phase, heart_rate, cramp_level, bbt):
    if cycle_phase == 1: # Menstruation
        if cramp_level >= 7 or bbt > 37.5:
            return 'Gentle yoga, stretching, walking'
        elif cramp_level <= 3 and bbt < 36.5:
            return 'Light cardio, walking, or strength training'
        else:
            return 'Yoga, light stretching'
    elif cycle_phase == 2: # Follicular
        if heart_rate < 70 and bbt < 36.5:
            return 'HIIT, running, strength training'
        else:
            return 'Moderate cardio, Pilates'
    elif cycle_phase == 3: # Ovulation
        if cramp_level <= 3 and heart_rate < 90:
            return 'High-intensity workouts, running, strength training'
        else:
            return 'Moderate cardio, cycling'
    elif cycle_phase == 4: # Luteal
        if bbt > 37.5 or heart_rate > 90:
            return 'Gentle yoga, swimming, stretching'
        else:
            return 'Moderate strength training, Pilates'
    return 'Rest'

In [4]:
def recommend_exercise_time(cycle_phase, heart_rate, cramp_level, bbt):
    if cycle_phase == 1: # Menstruation
        if cramp_level >= 7 or bbt > 37.5:
            return 10
        elif cramp_level <= 3 and bbt < 36.5:
            return 30
        else:
            return 20
    elif cycle_phase == 2: # Follicular
        if heart_rate < 70 and bbt < 36.5:
            return 45
        else:
            return 30
    elif cycle_phase == 3: # Ovulation
        if cramp_level <= 3 and heart_rate < 90:
            return 60
        else:
            return 30
    elif cycle_phase == 4: # Luteal
        if bbt > 37.5 or heart_rate > 90:
            return 20
        else:
            return 30
    return 0

In [5]:
df['Exercise in mins'] = df.apply(lambda row: recommend_exercise_time(row['Cycle Number'], row['Heart Rate'], row['Cramp Level'], row['Basal Body Temperature']), axis=1)
df['Exercise types'] = df.apply(lambda row: recommend_exercise_type(row['Cycle Number'], row['Heart Rate'], row['Cramp Level'], row['Basal Body Temperature']), axis=1)
df['Exercise types'] = df['Exercise types'].astype(str)

In [6]:
le = LabelEncoder()
df['Exercise types'] = le.fit_transform(df['Exercise types'])
X = df[['Cycle Number', 'Day Number', 'Cramp Level', 'Heart Rate', 'Basal Body Temperature']]
y = df[['Exercise in mins', 'Exercise types']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
regressor = LinearRegression()
classifier = RandomForestClassifier()
regressor.fit(X_train, y_train['Exercise in mins'])
classifier.fit(X_train, y_train['Exercise types'])
y_pred_mins = regressor.predict(X_test)
y_pred_types = classifier.predict(X_test)
mse_mins = mean_squared_error(y_test['Exercise in mins'], y_pred_mins)
accuracy_types = accuracy_score(y_test['Exercise types'], y_pred_types)

In [7]:
# Using the outputs generated by the neural network - SOMYA

cycle_number = 0
day_number = 0
cramp_level = 0
heart_rate = 0
bbt = 0

In [8]:
new_input = pd.DataFrame([[1, 7, 4, 72, 36.7]], columns=['Cycle Number', 'Day Number', 'Cramp Level', 'Heart Rate', 'Basal Body Temperature'])
new_pred_mins = math.ceil(regressor.predict(new_input)[0])
predicted_exercise_type = le.inverse_transform(classifier.predict(new_input))[0]